# Data Warehouse Utilization

In [1]:
%load_ext sql

In [2]:
%%sql
sqlite:///CourseDataWarehouse.db

'Connected: @CourseDataWarehouse.db'

## Which classroom is the most utilized?

Problem- each location is only counted once... we can't count popularity with distinct values in our database.... ask Huntley or rework Q?

In [22]:
%%sql
SELECT location,COUNT(*) as count 
FROM LOCATIONS_DW 
GROUP BY location 
ORDER BY count DESC
LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


location,count
BCC 200,1
BD,1
BH,1
BH BY ARR,1
BLM 112,1
BLM LL105,1
BNW 124,1
BNW 127,1
BNW 128,1
BNW 129B,1


## Which classes go over capacity the most frequently?

Problem- why are some professors teaching multiple sections of the same course? 6 sections of AC 0012 by same prof? check if accurate...

In [36]:
%%sql
SELECT COURSES_DW.CatalogYear, professor_id, COURSES_DW.Course_id, Catalog_id, Course_title, Timecodes, Cap, Actual, Remaining
FROM COURSES_DW    
    JOIN CLASS_FACTS_DW USING (course_id)
    JOIN TIMECODES_DW USING(Timecode_id)
    WHERE Remaining < 0 AND Cap != 0
    --GROUP BY course_title
    ORDER BY professor_id
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


CatalogYear,Professor_id,Course_id,Catalog_id,Course_title,Timecodes,Cap,Actual,Remaining
2017_2018,2,135,AC 0012,Introduction to Management Accounting,['MR 1100am-1215pm 01/16-05/01 DSB 108'],25,27,-2
2017_2018,2,135,AC 0012,Introduction to Management Accounting,['MR 1100am-1215pm 01/16-05/01 DSB 108'],25,27,-2
2017_2018,2,135,AC 0012,Introduction to Management Accounting,['MR 1100am-1215pm 01/16-05/01 DSB 108'],25,27,-2
2017_2018,2,135,AC 0012,Introduction to Management Accounting,['MR 1230pm-0145pm 01/16-05/01 DSB 108'],25,29,-4
2017_2018,2,135,AC 0012,Introduction to Management Accounting,['MR 1230pm-0145pm 01/16-05/01 DSB 108'],25,29,-4
2017_2018,2,135,AC 0012,Introduction to Management Accounting,['MR 1230pm-0145pm 01/16-05/01 DSB 108'],25,29,-4
2017_2018,3,161,AC 0330,Auditing,['MR 0200pm-0315pm 09/05-12/11 DSB 112'],26,27,-1
2017_2018,3,161,AC 0330,Auditing,['MR 0200pm-0315pm 09/05-12/11 DSB 112'],26,27,-1
2017_2018,3,161,AC 0330,Auditing,['MR 0330pm-0445pm 09/05-12/11 DSB 112'],26,27,-1
2017_2018,3,161,AC 0330,Auditing,['MR 0330pm-0445pm 09/05-12/11 DSB 112'],26,27,-1


In [ ]:
%%sql
--do not delete: original code of previous cell, save for reference
    
SELECT COURSES_DW.Course_id, Catalog_id, Course_title, Cap, Actual, Remaining, COUNT(course_id) AS Overflow_count
FROM COURSES_DW    
    JOIN CLASS_FACTS_DW USING (course_id)
    WHERE Remaining < 0 AND Cap != 0
    --GROUP BY CLASS_FACTS_DW.Course_id
    ORDER BY Overflow_count DESC
    LIMIT 20;

## Which professors have the most diverse courseload?

## On which day are the most/least classes held? (DONE)

In [8]:
%%sql
SELECT DISTINCT Day, count(Day) as Count
FROM TIMECODES_DW
GROUP BY Day
ORDER BY Count DESC;

 * sqlite:///CourseDataWarehouse.db
Done.


Day,Count
T,249
W,243
R,206
M,196
F,137
S,45
U,14


In [11]:
%%sql
SELECT COUNT(*)
FROM TIMECODES_DW
WHERE Start = '12:30:00';

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(*)
49


## Which professor has taught the most classes? students? (use actual, not cap)

Comment- double check if this is right. Might be over counting bc of same problem as first question

In [23]:
%%sql
SELECT Professor_id AS ID, program_name AS Program, Name, COUNT(Course_title) AS Courses_Taught
FROM PROFESSORS_DW    
    JOIN CLASS_FACTS_DW USING (professor_id)
    JOIN PROGRAMS_DW USING (program_id)
    JOIN COURSES_DW USING (course_id)
    GROUP BY ID
    ORDER BY Courses_Taught DESC
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


ID,Program,Name,Courses_Taught
984,Accounting,Victor E. Hartman,72
374,Nursing,Mary M Murphy,67
9,Accounting,Milo W. Peck,63
457,Physics,Bidyut B. Das,57
842,Spanish,Lauren S. Gaskill,56
383,Nursing,Laura A. Conklin,54
379,Nursing,Kelly Ann Sullivan,51
14,Accounting,Patricia M. Poli,47
251,Sociology,Rose P. Rodrigues,46
398,Nursing,Martha Ann Boulton,46


In [27]:
%%sql
SELECT Professor_id AS ID, program_name AS Program, Name, SUM(actual) AS Students_Taught
FROM PROFESSORS_DW    
    JOIN CLASS_FACTS_DW USING (professor_id)
    JOIN PROGRAMS_DW USING (program_id)
    JOIN COURSES_DW USING (course_id)
    GROUP BY ID
    ORDER BY Students_Taught DESC
    LIMIT 10;

 * sqlite:///CourseDataWarehouse.db
Done.


ID,Program,Name,Students_Taught
374,Nursing,Mary M Murphy,3249
380,Nursing,Katherine Marie Saracino,1611
9,Accounting,Milo W. Peck,1598
51,Biology,Christine Earls,1319
52,Biology,Anthony J. DeCristofaro,1190
457,Physics,Bidyut B. Das,1121
307,Marketing,John E. Neal,1110
984,Accounting,Victor E. Hartman,1077
297,Management,Paul P. Daulerio,1058
496,Religious Studies,John E. Thiel,1008


## Which classes are attracting >10 students? (which classes are the least popular--- ratio?)